## 問題

https://twitter.com/e869120/status/1383189464650981378

## 解説

https://twitter.com/e869120/status/1383635232134287360

In [1]:
#r "nuget: FSharpPlus"

Installed Packages FSharpPlus, 1.3.2

In [2]:
open System
open System.Numerics
open System.Collections.Generic
open FSharpPlus

In [12]:
let solveUsingFullSearch N Ms =
  let Ms = Ms |> Seq.toArray
  seq { 0 .. Ms.Length - 2 } |> Seq.sumBy (fun s ->
    seq { s + 1 .. Ms.Length - 1 } |> Seq.filter (fun t ->
      let (ls, rs), (lt, rt) = Ms[s], Ms[t]
      ls < lt && lt < rs && rs < rt || lt < ls && ls < rt && rt < rs)
    |> Seq.length)

In [13]:
solveUsingFullSearch 6 [2, 5; 1, 4; 1, 3]

2

In [14]:
solveUsingFullSearch 250 [
  13, 218
  17, 99
  24, 180
  53, 115
  96, 97
  111, 158
  124, 164
  135, 227
  158, 177
  204, 224
]

10

In [15]:
solveUsingFullSearch 100 [
  1, 2
  1, 3
  1, 4
  1, 5
  1, 6
  1, 7
  1, 8
  1, 9
  1, 10
  1, 11
]

0

In [16]:
solveUsingFullSearch 100 [
  1, 100
  2, 99
  3, 98
  4, 97
  5, 96
  6, 95
  7, 94
  8, 93
  9, 92
  10, 91
]

0

In [17]:
solveUsingFullSearch 1000 [
  12, 43
  23, 59
  32, 118
  44, 751
  68, 136
  70, 168
  85, 328
  88, 809
  92, 981
  95, 540
  98, 772
  98, 903
  125, 896
  173, 737
  199, 325
  212, 369
  227, 587
  230, 374
  287, 442
  306, 926
  314, 858
  316, 371
  318, 493
  337, 506
  384, 887
  387, 493
  394, 457
  404, 652
  414, 527
  422, 920
  441, 730
  445, 620
  468, 602
  482, 676
  568, 857
  587, 966
  653, 757
  710, 928
  764, 927
  778, 916
]

229

In [20]:
let calcUsingIncident N Ms =
  let countMs = Ms |> List.length |> int64
  let allPatternCount = countMs * (countMs - 1L) / 2L
  // 各端点に属する線分の数を求めておく
  let countsL = Ms |> Seq.map fst |> Seq.countBy id |> dict
  let countsR = Ms |> Seq.map snd |> Seq.countBy id |> dict
  let getCount i (counts: IDictionary<int, int>) = counts.TryGetValue(i) |> Option.ofPair |> Option.defaultValue 0 |> int64
  /// 端点で交わる場合の数
  let result1 =
    seq { 1 .. N } |> Seq.sumBy (fun i ->
      let c = getCount i countsL + getCount i countsR
      c * (c - 1L) / 2L)
  /// 線分が離れている場合の数
  let result2 =
    let counts = (0L, seq { 1 .. N }) ||> Seq.scan (fun s i -> s + getCount i countsR) |> Seq.toArray
    Ms |> Seq.sumBy (fun (l, _) -> counts[l - 1])
  /// 線分がもう1つの線分を包含している場合の数
  let result3 =
    ((0L, Array.zeroCreate<int64> (N + 1)), Ms |> Seq.sortBy (fun (l, r) -> r, l)) ||> Seq.fold (fun (result, counts) (l, r) ->
      let result = result + (seq { l + 1 .. r - 1 } |> Seq.sumBy (fun i -> counts[i]))
      counts[l] <- counts[l] + 1L
      result, counts)
    |> fst
  allPatternCount - result1 - result2 - result3

In [26]:
calcUsingIncident 6 [2, 5; 1, 4; 1, 3]

2

In [27]:
calcUsingIncident 250 [
  13, 218
  17, 99
  24, 180
  53, 115
  96, 97
  111, 158
  124, 164
  135, 227
  158, 177
  204, 224
]

10

In [28]:
calcUsingIncident 100 [
  1, 2
  1, 3
  1, 4
  1, 5
  1, 6
  1, 7
  1, 8
  1, 9
  1, 10
  1, 11
]

0

In [29]:
calcUsingIncident 100 [
  1, 100
  2, 99
  3, 98
  4, 97
  5, 96
  6, 95
  7, 94
  8, 93
  9, 92
  10, 91
]

0

In [30]:
calcUsingIncident 1000 [
  12, 43
  23, 59
  32, 118
  44, 751
  68, 136
  70, 168
  85, 328
  88, 809
  92, 981
  95, 540
  98, 772
  98, 903
  125, 896
  173, 737
  199, 325
  212, 369
  227, 587
  230, 374
  287, 442
  306, 926
  314, 858
  316, 371
  318, 493
  337, 506
  384, 887
  387, 493
  394, 457
  404, 652
  414, 527
  422, 920
  441, 730
  445, 620
  468, 602
  482, 676
  568, 857
  587, 966
  653, 757
  710, 928
  764, 927
  778, 916
]

229

In [43]:
type BinaryIndexedTree(count) =
  let bit = Array.zeroCreate<int64> (count + 1)
  member _.add i v =
    let rec loop i =
      if i < bit.Length then
        bit[i] <- bit[i] + v
        loop (i + (i &&& -i))
    loop i
  member _.sum r =
    let rec loop result i =
      if 0 < i then loop (result + bit[i]) (i - (i &&& -i))
      else result
    loop 0L r
  member this.sum(l, r) = this.sum r - this.sum (l - 1)
  new(xs: int64[]) as this =
    BinaryIndexedTree(xs.Length)
    then
      for i = 0 to xs.Length - 1 do this.add (i + 1) xs[i]

In [52]:
let calcUsingBIT N Ms =
  // 途中までは calcUsingIncident と同じ
  let countMs = Ms |> List.length |> int64
  let allPatternCount = countMs * (countMs - 1L) / 2L
  let countsL = Ms |> Seq.map fst |> Seq.countBy id |> dict
  let countsR = Ms |> Seq.map snd |> Seq.countBy id |> dict
  let getCount i (counts: IDictionary<int, int>) = counts.TryGetValue(i) |> Option.ofPair |> Option.defaultValue 0 |> int64
  let result1 =
    seq { 1 .. N } |> Seq.sumBy (fun i ->
      let c = getCount i countsL + getCount i countsR
      c * (c - 1L) / 2L)
  let result2 =
    let counts = (0L, seq { 1 .. N }) ||> Seq.scan (fun s i -> s + getCount i countsR) |> Seq.toArray
    Ms |> Seq.sumBy (fun (l, _) -> counts[l - 1])
  /// 線分がもう1つの線分を包含している場合の数をBinary Indexed Treeを使って高速に求める
  let result3 =
    let bit = BinaryIndexedTree(N)
    Ms |> Seq.sortBy (fun (l, r) -> r, l) |> Seq.sumBy (fun (l, r) ->
      let result = bit.sum(l + 1, r - 1)
      bit.add l 1
      result)
  allPatternCount - result1 - result2 - result3

In [53]:
calcUsingBIT 6 [2, 5; 1, 4; 1, 3]

2

In [54]:
calcUsingBIT 250 [
  13, 218
  17, 99
  24, 180
  53, 115
  96, 97
  111, 158
  124, 164
  135, 227
  158, 177
  204, 224
]

10

In [55]:
calcUsingBIT 100 [
  1, 2
  1, 3
  1, 4
  1, 5
  1, 6
  1, 7
  1, 8
  1, 9
  1, 10
  1, 11
]

0

In [56]:
calcUsingBIT 100 [
  1, 100
  2, 99
  3, 98
  4, 97
  5, 96
  6, 95
  7, 94
  8, 93
  9, 92
  10, 91
]

0

In [57]:
calcUsingBIT 1000 [
  12, 43
  23, 59
  32, 118
  44, 751
  68, 136
  70, 168
  85, 328
  88, 809
  92, 981
  95, 540
  98, 772
  98, 903
  125, 896
  173, 737
  199, 325
  212, 369
  227, 587
  230, 374
  287, 442
  306, 926
  314, 858
  316, 371
  318, 493
  337, 506
  384, 887
  387, 493
  394, 457
  404, 652
  414, 527
  422, 920
  441, 730
  445, 620
  468, 602
  482, 676
  568, 857
  587, 966
  653, 757
  710, 928
  764, 927
  778, 916
]

229